In [1]:
from data.image_pair_dataset import Paired_Dataset
from torchvision.transforms import transforms


data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
data = Paired_Dataset(image_directories=data_dirs, scale=2, patch_size=(500, 500))

hr, lr = data[470]
print(hr.shape, lr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([3, 1000, 1000]) torch.Size([3, 500, 500])


In [2]:
from data.get_loader import get_loader
from torchvision.transforms import transforms

data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
prefetcher = get_loader(image_directories=data_dirs,
                        scale=4,
                        patch_size=(100, 100),
                        batch_size=1,
                        device=None,
                        num_workers=2,
                        pin_memory=False)


batch = prefetcher.next()
lr, hr = batch
print(lr.shape, hr.shape)
lr = lr.squeeze(0)
hr = hr.squeeze(0)
print(lr.shape, hr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([1, 3, 100, 100]) torch.Size([1, 3, 400, 400])
torch.Size([3, 100, 100]) torch.Size([3, 400, 400])


In [1]:
from train.trainer import Trainer

trainer = Trainer('train_yamls/RWVKSR_2X.yaml')
trainer.train()

config loaded successfully
data prefetcher using device: cuda
data prefetcher initialized successfully
dataloader created successfully
optimizer: adam initialized successfully
using cuda autocast with dtype: torch.bfloat16


wandb: Currently logged in as: farns (farns301) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb initialized successfully
model has 1191436 trainable parameters
device used for training: cuda
starting training...


Epoch 1/300:   0%|          | 0/54 [00:00<?, ?it/s]

torch.Size([64, 3, 96, 96]) torch.Size([64, 3, 48, 48])


c:\Users\farns\miniforge3\envs\pytorch\Lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([64, 3, 48, 48])) that is different to the input size (torch.Size([64, 3, 192, 192])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
Epoch 1/300:   0%|          | 0/54 [00:01<?, ?it/s]


RuntimeError: The size of tensor a (192) must match the size of tensor b (48) at non-singleton dimension 3

epoch evaluated: 20


C:\Users\farns\AppData\Local\Temp\ipykernel_27728\1562054747.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('checkpoints/2X_RWKVSR/epoch_20.pt')